In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('chroma_dataset.csv')
df

,Unnamed: 0,pdb,split
0,0,12as,train
1,1,12e8,train
2,2,148l,train
3,3,16pk,train
4,4,16vp,train
...,...,...,...
28814,28814,7tt9,validation
28815,28815,7v1q,validation
28816,28816,7vem,validation
28817,28817,7w83,validation


In [26]:
from collections import defaultdict
from moleculib.assembly.datum import AssemblyDatum
from tqdm.contrib.concurrent import process_map


def process(row):
    _, row = row
    try:
        assembly = AssemblyDatum.fetch_pdb_id(row.pdb)
    except:
        return None
    return [row.split, assembly]

data = process_map(process, list(df.iterrows()), max_workers=25)


/tmp/ipykernel_337244/392068851.py:14: TqdmWarning: Iterable length 28819 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  data = process_map(process, list(df.iterrows()), max_workers=25)
100%|██████████| 28819/28819 [05:34<00:00, 86.06it/s] 


In [7]:
data[0][1].protein_data[0]

In [27]:

dataset = defaultdict(list)
for row in data:
    if row is None:
        continue
    split, assembly = row
    dataset[split].append(assembly)

In [22]:
dataset['train'][10].protein_data

In [28]:
save_path = '/mas/projects/molecularmachines/db/PREPROCESSED/CHROMA_FULL.pyd'

import pickle
with open(save_path, 'wb') as f:
    pickle.dump(dataset, f)

In [24]:
dataset['train']

In [31]:
dataset_reduced = { 'train': dataset['train'][:500] }
with open(save_path.replace('FULL', 'TINY'), 'wb') as f:
    pickle.dump(dataset_reduced, f)

In [30]:
dataset['train'][0].protein_data[0].idcode

'12asB'

In [24]:
assembly.idcode

'12as'

In [23]:
from moleculib.graphics.py3Dmol import plot_py3dmol_grid, plot_py3dmol
plot_py3dmol([assembly.protein_data[0], assembly.protein_data[1]])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol